In [1]:
import wrds
import pandas as pd
from sqlalchemy import create_engine

In [2]:
start_date = '2020-01-01'
end_date = '2023-12-31'
db_path = '/Users/niveditavats/Documents/Projects_CC/equity_analytics_pipelines/database/investment_data.sqlite'


In [3]:
conn = wrds.Connection()
engine = create_engine(f"sqlite:///{db_path}")


Loading library list...
Done


In [4]:
# Load permnos from compustat_quarterly table
permnos = pd.read_sql("SELECT DISTINCT permno FROM compustat_quarterly", con=engine)['permno'].dropna().astype(int).tolist()
permnos_str = ','.join(map(str, permnos))

In [5]:
query = f"""
    SELECT permno, date, ret
    FROM crsp.dsf
    WHERE date BETWEEN '{start_date}' AND '{end_date}'
      AND permno IN (14593)  -- Example: AAPL permno, replace with mapped permnos
"""

In [6]:
prices = conn.raw_sql(query)
prices['symbol'] = prices['permno'].astype(str)
prices['date'] = pd.to_datetime(prices['date'])
prices = prices[prices['ret'].notnull()]
prices.to_sql("crsp_daily", con=engine, if_exists="replace", index=False)
print("CRSP daily returns pulled for mapped permnos and stored in SQLite.")


CRSP daily returns pulled for mapped permnos and stored in SQLite.
